## Random Forest

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 🔹 STEP 1: Load the dataset
file_path = "Updated_MindEase_Cleaned.csv"  # Change this to your actual path
df = pd.read_csv(file_path)

# 🔹 STEP 2: Define features (X) and target (y)
target_column = "Mental_Health_Condition"
X = df.drop(columns=[target_column])
y = df[target_column]

# 🔹 STEP 3: Convert target column (y) to numerical labels
y = y.astype(str)  # Ensure consistency
y = pd.factorize(y)[0]  # Convert categorical target to numbers

# 🔹 STEP 4: Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Check for missing columns
remaining_cols = set(X.columns) - set(categorical_cols) - set(numerical_cols)

# Convert any non-classified columns
for col in remaining_cols:
    try:
        X[col] = pd.to_numeric(X[col], errors="coerce")
        numerical_cols.append(col)
    except:
        categorical_cols.append(col)

# Ensure all features are accounted for
assert len(categorical_cols) + len(numerical_cols) == X.shape[1], "Mismatch in feature selection!"

# 🔹 STEP 5: Create preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ("num", SimpleImputer(strategy="mean"), numerical_cols),  # Fill missing numerical values with mean
    ("cat", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),  # Fill missing categorical values
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # One-hot encode categorical features
    ]), categorical_cols)
])

# 🔹 STEP 6: Split the dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🔹 STEP 7: Train the Random Forest Model
rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42))
])

rf_model.fit(X_train, y_train)

# 🔹 STEP 8: Predict and evaluate
y_pred_rf = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"🌲 Random Forest Accuracy: {rf_accuracy * 100:.2f}%")

# 🔹 STEP 9: Fine-Tune Random Forest with RandomizedSearchCV
param_grid_rf = {
    "classifier__n_estimators": [100, 200, 300, 400],
    "classifier__max_depth": [5, 10, 20, 30],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4]
}

random_search_rf = RandomizedSearchCV(rf_model, param_distributions=param_grid_rf, n_iter=20,
                                      cv=3, scoring="accuracy", n_jobs=-1, random_state=42, verbose=2)
random_search_rf.fit(X_train, y_train)

# 🔹 STEP 10: Get Best Random Forest Parameters & Accuracy
best_rf = random_search_rf.best_estimator_
best_rf_accuracy = accuracy_score(y_test, best_rf.predict(X_test))
print(f" Best Random Forest Accuracy after Tuning: {best_rf_accuracy * 100:.2f}%")
print(f" Best Random Forest Parameters: {random_search_rf.best_params_}")

# 🔹 STEP 11: Feature Importance Analysis
importances = best_rf.named_steps["classifier"].feature_importances_
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()
feature_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

print("\n Top 10 Important Features:")
print(feature_importance_df.head(10))


🌲 Random Forest Accuracy: 88.33%
Fitting 3 folds for each of 20 candidates, totalling 60 fits
 Best Random Forest Accuracy after Tuning: 87.78%
 Best Random Forest Parameters: {'classifier__n_estimators': 200, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 20}

 Top 10 Important Features:
                      Feature  Importance
0                    num__Age    0.141677
4               num__Duration    0.139326
2                   num__Mood    0.082914
9   num__Mood_Before_Activity    0.082552
3          num__Activity_Type    0.082174
10   num__Mood_After_Activity    0.077908
5          num__Exercise_Type    0.076724
7                   num__Goal    0.073336
8           num__Support_Type    0.065524
1                 num__Gender    0.051705


In [2]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# knn = KNeighborsClassifier(n_neighbors=5)

# knn.fit(X_train,y_train)

# y_pred = knn.predict(X_test)

# accuracy_score(y_test,y_pred)

In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_csv("Updated_MindEase_Cleaned.csv")  # Replace with actual dataset path

# Encode categorical features
label_encoders = {}
categorical_cols = ["Gender", "Therapist_Interaction", "Sleep_Category"]

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store encoders for later use

# Encode target variable
target_encoder = LabelEncoder()
data["Mental_Health_Condition"] = target_encoder.fit_transform(data["Mental_Health_Condition"])

# Split features and target
X = data.drop(columns=["Mental_Health_Condition"])
y = data["Mental_Health_Condition"]

# Train model
model = RandomForestClassifier()
model.fit(X, y)

# Save model and encoders
with open("mind_ease_model.pkl", "wb") as f:
    pickle.dump(model, f)
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)
with open("target_encoder.pkl", "wb") as f:
    pickle.dump(target_encoder, f)


In [7]:
# from flask import Flask, request, jsonify
# import numpy as np
# import pickle

# # Load trained model and encoders
# model = pickle.load(open("mind_ease_model.pkl", "rb"))
# label_encoders = pickle.load(open("label_encoders.pkl", "rb"))
# target_encoder = pickle.load(open("target_encoder.pkl", "rb"))

# app = Flask(__name__)

# @app.route("/")
# def index():
#     return "Hello world"

# @app.route("/predict", methods=["POST"])
# def predict():
#     # Extract input values
#     input_data = {
#         "Age": request.form.get("Age"),
#         "Gender": request.form.get("Gender"),
#         "Mood": request.form.get("Mood"),
#         "Activity_Type": request.form.get("Activity_Type"),
#         "Duration": request.form.get("Duration"),
#         "Exercise_Type": request.form.get("Exercise_Type"),
#         "Diet_Type": request.form.get("Diet_Type"),
#         "Goal": request.form.get("Goal"),
#         "Support_Type": request.form.get("Support_Type"),
#         "Mood_Before_Activity": request.form.get("Mood_Before_Activity"),
#         "Mood_After_Activity": request.form.get("Mood_After_Activity"),
#         "Therapist_Interaction": request.form.get("Therapist_Interaction"),
#         "Sleep_Category": request.form.get("Sleep_Category"),
#     }

#     # Convert to DataFrame
#     input_df = np.array([list(input_data.values())], dtype=object)

#     # Apply label encoding to categorical features
#     categorical_cols = ["Gender", "Therapist_Interaction", "Sleep_Category"]
#     for i, col in enumerate(categorical_cols):
#         input_df[0][i] = label_encoders[col].transform([input_df[0][i]])[0]

#     # Convert all values to float for model
#     input_df = input_df.astype(float)

#     # Predict
#     prediction_numeric = model.predict(input_df)[0]

#     # Convert prediction back to label
#     prediction_label = target_encoder.inverse_transform([prediction_numeric])[0]

#     return jsonify({"Mental_Health_Condition": prediction_label})

# if __name__ == "__main__":
#     app.run(debug=True)



# from flask import Flask, request, jsonify
# import numpy as np
# import pickle
# from flask_cors import CORS  # Allow API access from different origins

# # Initialize Flask app
# app = Flask(__name__)
# CORS(app)  # Enable CORS

# # Load the trained model safely
# try:
#     model = pickle.load(open('mind_ease_model.pkl', 'rb'))
#     if not hasattr(model, "predict"):
#         raise ValueError("Loaded model is invalid. It does not have a predict() method.")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     model = None  # Avoid using an invalid model

# @app.route('/')
# def index():
#     return "Hello world - Mind Ease API is running!"

# @app.route('/predict', methods=['POST'])
# def predict():
#     if not model:
#         return jsonify({"error": "Model not loaded properly. Check your 'mind_ease_model.pkl' file."}), 500

#     # Ensure JSON data is received
#     data = request.get_json(silent=True)
#     if not data:
#         return jsonify({"error": "Invalid input. Please send JSON data."}), 400

#     # Debugging: Print received data
#     print("Received Data:", data)

#     try:
#         # Extract input values, handle missing values with defaults
#         Age = float(data.get('Age', 0))
#         Gender = int(data.get('Gender', 0))
#         Mood = int(data.get('Mood', 0))
#         Activity_Type = int(data.get('Activity_Type', 0))
#         Duration = float(data.get('Duration', 0))
#         Exercise_Type = int(data.get('Exercise_Type', 0))
#         Diet_Type = int(data.get('Diet_Type', 0))
#         Goal = int(data.get('Goal', 0))
#         Support_Type = int(data.get('Support_Type', 0))
#         Mood_Before_Activity = int(data.get('Mood_Before_Activity', 0))
#         Mood_After_Activity = int(data.get('Mood_After_Activity', 0))

#         # Convert categorical data
#         Therapist_Interaction = 1 if data.get('Therapist_Interaction') == "Yes" else 0
#         Sleep_Category = {"Low": 0, "Moderate": 1, "High": 2}.get(data.get('Sleep_Category'), -1)

#         # Prepare input array for model
#         input_query = np.array([[Age, Gender, Mood, Activity_Type, Duration, Exercise_Type,
#                                  Diet_Type, Goal, Support_Type, Mood_Before_Activity,
#                                  Mood_After_Activity, Therapist_Interaction, Sleep_Category]])

#         # Debugging: Print model input
#         print("Model Input:", input_query)

#         # Make prediction
#         prediction = model.predict(input_query)[0]

#         # Return the prediction as JSON response
#         return jsonify({"Mental_Health_Condition": str(prediction)})

#     except Exception as e:
#         print(f"Error processing request: {e}")
#         return jsonify({"error": str(e)}), 500

# # Run Flask server
# if __name__ == '__main__':
#     app.run(debug=True)




# from flask import Flask, request, jsonify
# from flask_cors import CORS
# import numpy as np
# import pickle

# # Initialize Flask app
# app = Flask(__name__)
# CORS(app)  # Enable CORS to allow cross-origin requests

# # Load the trained model safely
# try:
#     model = pickle.load(open('mind_ease_model.pkl', 'rb'))
#     if not hasattr(model, "predict"):
#         raise ValueError("Loaded model is invalid. It does not have a predict() method.")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     model = None  # Prevent the app from crashing

# @app.route('/')
# def index():
#     return "Mental Health Prediction API is running."

# @app.route('/predict', methods=['POST'])
# def predict():
#     if model is None:
#         return jsonify({'error': "Model not loaded correctly. Check 'mind_ease_model.pkl' file."}), 500

#     try:
#         data = request.get_json()  # Get JSON data from Postman

#         # Convert categorical values properly
#         Age = float(data.get('Age', 0))
#         Gender = int(data.get('Gender', 0))
#         Mood = int(data.get('Mood', 0))
#         Activity_Type = int(data.get('Activity_Type', 0))
#         Duration = float(data.get('Duration', 0))
#         Exercise_Type = int(data.get('Exercise_Type', 0))
#         Diet_Type = int(data.get('Diet_Type', 0))
#         Goal = int(data.get('Goal', 0))
#         Support_Type = int(data.get('Support_Type', 0))
#         Sleep_Category = data.get('Sleep_Category', 'Moderate')  # Default to 'Moderate'

#         # Encode Sleep_Category (if your model requires numerical values)
#         sleep_mapping = {"Low": 0, "Moderate": 1, "High": 2}
#         Sleep_Category = sleep_mapping.get(Sleep_Category, 1)

#         # Prepare input data
#         input_query = np.array([[Age, Gender, Mood, Activity_Type, Duration, Exercise_Type, Diet_Type, Goal, Support_Type, Sleep_Category]])

#         # Make prediction
#         result = model.predict(input_query)[0]

#         return jsonify({'Mental_Health_Condition': str(result)})

#     except Exception as e:
#         return jsonify({'error': str(e)}), 400

# if __name__ == '__main__':
#     app.run(debug=True)


from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import pickle
import os

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# ✅ Load the trained model safely
model_path = "mind_ease_model.pkl"

if not os.path.exists(model_path):
    print("❌ ERROR: Model file not found. Ensure 'mind_ease_model.pkl' exists.")
    exit(1)

try:
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    # Ensure the model has a 'predict' method
    if not hasattr(model, "predict"):
        print("❌ ERROR: Loaded model is invalid. It does not have a 'predict' method.")
        exit(1)

    print("✅ Model loaded successfully!")

except Exception as e:
    print(f"❌ ERROR loading model: {e}")
    exit(1)

@app.route('/')
def home():
    return "Mental Health Prediction API is Running!"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()

        # Required fields
        required_fields = [
            "Age", "Gender", "Mood", "Activity_Type", "Duration",
            "Exercise_Type", "Diet_Type", "Goal", "Support_Type",
            "Mood_Before_Activity", "Mood_After_Activity",
            "Therapist_Interaction", "Sleep_Category"
        ]
        for field in required_fields:
            if field not in data:
                return jsonify({"error": f"Missing field: {field}"}), 400

        # Convert categorical data
        data["Therapist_Interaction"] = 1 if data["Therapist_Interaction"].lower() == "yes" else 0
        sleep_mapping = {"Low": 0, "Moderate": 1, "High": 2}
        data["Sleep_Category"] = sleep_mapping.get(data["Sleep_Category"], -1)

        # Convert data to numpy array
        input_data = np.array([
            data["Age"], data["Gender"], data["Mood"], data["Activity_Type"],
            data["Duration"], data["Exercise_Type"], data["Diet_Type"],
            data["Goal"], data["Support_Type"], data["Mood_Before_Activity"],
            data["Mood_After_Activity"], data["Therapist_Interaction"],
            data["Sleep_Category"]
        ]).reshape(1, -1)

        # Predict
        prediction = model.predict(input_data)[0]

        return jsonify({"Mental_Health_Condition": str(prediction)})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


✅ Model loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [ ]:
import joblib

# Save the best model
joblib.dump(best_rf, "mind_ease_model.pkl")
print("✅ Model saved as mind_ease_model.pkl")

✅ Model saved as mind_ease_model.pkl


In [8]:
import pandas as pd
import joblib

# Load the trained model
model_path = "mind_ease_model.pkl"  # Ensure this path is correct
model = joblib.load(model_path)

# Define the feature names required for model prediction
selected_features = ["Age", "Gender", "Mood_Before_Activity", "Mood", "Mood_After_Activity",
                     "Activity_Type", "Exercise_Type", "Duration", "Goal", "Diet_Type",
                     "Sleep_Category", "Support_Type", "Therapist_Interaction"]

# Mapping categorical values to numeric
mappings = {
    "Gender": {"Female": 0, "Non-binary": 1, "Male": 2, "No": 3},
    "Mood": {"Happy": 0, "Calm": 1, "Tired": 2, "Sad": 3, "Stressed": 4, "Anxious": 5, "Excited": 6},
    "Mood_Before_Activity": {"Tired": 0, "Happy": 1, "Sad": 2, "Calm": 3, "Stressed": 4, "Anxious": 5, "Excited": 6},
    "Mood_After_Activity": {"Happy": 0, "Tired": 1, "Anxious": 2, "Stressed": 3, "Calm": 4, "Sad": 5, "Excited": 6},
    "Activity_Type": {"Therapy": 0, "Journaling": 1, "Yoga": 2, "Meditation": 3, "Breathing Exercise": 4, "Walking": 5, "Running": 6},
    "Exercise_Type": {"Walking": 0, "None": 1, "Running": 2, "Yoga": 3, "Swimming": 4, "Cycling": 5},
    "Goal": {"Reduce Stress": 0, "Better Sleep": 1, "Improve Focus": 2, "Manage Anxiety": 3, "Boost Energy": 4},
    "Diet_Type": {"Balanced": 0, "Healthy": 1, "Junk": 2},
    "Support_Type": {"Sleep Music": 0, "CBT (Cognitive Behavioral Therapy)": 1, "Mood Tracking": 2, "Therapy": 3, "Meditation": 4, "None": 5},
    "Therapist_Interaction": {"Yes": 0, "No": 1},
}

# Function to take user input and convert categorical values to numeric
def get_user_input():
    user_data = {}

    for feature in selected_features:
        if feature in mappings:
            valid_options = list(mappings[feature].keys())
            print(f"\n{feature} Options: {valid_options}")  # Show valid options
            value = input(f"Enter {feature} (choose from above options): ").strip()

            if value in mappings[feature]:
                user_data[feature] = mappings[feature][value]
            else:
                print(f"❌ Invalid input for {feature}. Please enter one of {valid_options}.")
                return None  # Stop execution if invalid input is entered
        else:
            while True:
                try:
                    value = float(input(f"Enter {feature} (numeric value): ").strip())
                    user_data[feature] = value
                    break
                except ValueError:
                    print(f"❌ Invalid input for {feature}. Please enter a valid number.")

    return pd.DataFrame([user_data])

# Function to recommend a mindfulness activity based on user mood
def recommend_activity(mood):
    recommendations = {
        "Happy": "Meditation to maintain positive energy 😊",
        "Calm": "Yoga for continued relaxation 🧘‍♂️",
        "Tired": "Breathing exercises to refresh yourself 😴",
        "Sad": "Journaling to express your thoughts 📖",
        "Stressed": "Therapy or mindfulness meditation to ease stress 🌿",
        "Anxious": "Cognitive Behavioral Therapy (CBT) techniques for anxiety 🧠",
        "Excited": "Engage in physical activities like running or cycling 🚴",
    }
    return recommendations.get(mood, "Try some relaxation exercises!")

# Get user input
user_input_df = get_user_input()

if user_input_df is not None:
    # Ensure input matches model's feature format
    user_input_df = user_input_df.reindex(columns=selected_features, fill_value=0)

    # Predict mental health condition
    predicted_label = model.predict(user_input_df)[0]

    # Convert numeric output to categorical label
    mental_health_mapping = {0: "None", 1: "Anxiety", 2: "Depression", 3: "Stress"}
    predicted_condition = mental_health_mapping.get(predicted_label, "Unknown")

    # Get recommendation based on mood
    mood_input = list(mappings["Mood"].keys())[list(mappings["Mood"].values()).index(user_input_df["Mood"].values[0])]
    recommended_activity = recommend_activity(mood_input)

    print(f"\n🧠 Predicted Mental Health Condition: {predicted_condition}")
    print(f"💡 Recommended Activity: {recommended_activity}")



Gender Options: ['Female', 'Non-binary', 'Male', 'No']

Mood_Before_Activity Options: ['Tired', 'Happy', 'Sad', 'Calm', 'Stressed', 'Anxious', 'Excited']

Mood Options: ['Happy', 'Calm', 'Tired', 'Sad', 'Stressed', 'Anxious', 'Excited']

Mood_After_Activity Options: ['Happy', 'Tired', 'Anxious', 'Stressed', 'Calm', 'Sad', 'Excited']

Activity_Type Options: ['Therapy', 'Journaling', 'Yoga', 'Meditation', 'Breathing Exercise', 'Walking', 'Running']

Exercise_Type Options: ['Walking', 'None', 'Running', 'Yoga', 'Swimming', 'Cycling']

Goal Options: ['Reduce Stress', 'Better Sleep', 'Improve Focus', 'Manage Anxiety', 'Boost Energy']
❌ Invalid input for Goal. Please enter one of ['Reduce Stress', 'Better Sleep', 'Improve Focus', 'Manage Anxiety', 'Boost Energy'].


## For converting into .h5 format

In [ ]:
# import joblib
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Input

# # Save the best Random Forest model
# joblib.dump(best_rf, "mind_ease_model.pkl")

# # 🔹 Get the correct input shape by transforming a sample from the dataset
# X_sample = X_train.iloc[:1]  # Take a small sample from the training data
# X_transformed = best_rf.named_steps["preprocessor"].transform(X_sample)
# input_shape = X_transformed.shape[1]  # Extract the correct number of features

# # 🔹 Define a Neural Network (MLP) with the same input features
# mlp_model = Sequential([
#     Input(shape=(input_shape,)),
#     Dense(128, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(1, activation='sigmoid')  # Assuming binary classification
# ])

# # 🔹 Compile the model
# mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # 🔹 Save the model as .h5
# mlp_model.save("mind_ease_model.h5")
# print("✅ Model converted to .h5 format successfully!")


## Convert Model into Tflite

In [ ]:
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("mind_ease_model.h5")

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model converted to TFLite!")

INFO:tensorflow:Assets written to: C:\Users\Utkarsh\AppData\Local\Temp\tmpa0ifa5oc\assets


INFO:tensorflow:Assets written to: C:\Users\Utkarsh\AppData\Local\Temp\tmpa0ifa5oc\assets


Saved artifact at 'C:\Users\Utkarsh\AppData\Local\Temp\tmpa0ifa5oc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 16), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2387333353872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333356560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333357712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333356176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333358096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333358672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333359248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2387333360016: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted to TFLite!


# Steps to Deploy Your ML Model in an Android App (Java)
# 1. Convert your trained TensorFlow model to .tflite.
# 2. Add TensorFlow Lite dependencies in build.gradle.
# 3. Place your model.tflite file in the assets/ folder.
# 4. Load the model and run inference in Java.

In [ ]:
# 2️⃣ Add Dependencies in Android Project
# Modify app/build.gradle:

dependencies {
    implementation 'org.tensorflow:tensorflow-lite:2.9.0'  // Add TensorFlow Lite library
}


In [ ]:
# 3️⃣ Place .tflite Model in assets/ Folder
Copy model.tflite and paste it into the assets/ folder in your Android project.
If the assets folder doesn’t exist, create one under src/main/

# 4. Load the Model

In [ ]:
# 4️⃣ Load and Run the TFLite Model in Java
## Create a Java class (MLModel.java) to handle TensorFlow Lite inference.

import android.content.Context;
import org.tensorflow.lite.Interpreter;
import java.io.File;
import java.io.FileInputStream;
import java.io.IOException;
import java.nio.MappedByteBuffer;
import java.nio.channels.FileChannel;
import java.nio.ByteBuffer;

public class MLModel {
    private Interpreter interpreter;

    // Constructor to load the model
    public MLModel(Context context) {
        try {
            interpreter = new Interpreter(loadModelFile(context));
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    // Load the .tflite model from assets
    private MappedByteBuffer loadModelFile(Context context) throws IOException {
        File file = new File(context.getFilesDir(), "model.tflite");
        FileInputStream inputStream = new FileInputStream(file);
        FileChannel fileChannel = inputStream.getChannel();
        return fileChannel.map(FileChannel.MapMode.READ_ONLY, 0, file.length());
    }

    // Run inference
    public float[] runModel(float[] inputData) {
        float[][] output = new float[1][1];  // Change output shape based on your model
        interpreter.run(inputData, output);
        return output[0];  // Return output data
    }
}

# 5️⃣ Use the Model in Your Android Activity
# Modify MainActivity.java to integrate the ML model.

In [ ]:
import android.os.Bundle;
import android.widget.TextView;
import androidx.appcompat.app.AppCompatActivity;

public class MainActivity extends AppCompatActivity {
    private MLModel mlModel;
    private TextView resultText;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        resultText = findViewById(R.id.result_text);

        // Load ML model
        mlModel = new MLModel(this);

        // Example input data (Modify based on your model's input)
        float[] inputData = {1.0f};

        // Run inference and get result
        float[] result = mlModel.runModel(inputData);

        // Display the output
        resultText.setText("Model Output: " + result[0]);
    }
}

# 6️⃣ Update UI Layout (activity_main.xml)

In [ ]:
# Modify res/layout/activity_main.

<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    android:gravity="center"
    android:padding="20dp">

    <TextView
        android:id="@+id/result_text"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Waiting for Model Output..."
        android:textSize="18sp"
        android:textColor="#000000"
        android:gravity="center"/>

</LinearLayout>


In [ ]:
# 7️⃣ Run the App
Ensure you have a valid .tflite model inside the assets/ folder.
Connect a device or use an emulator.
Run the app:
bash
Copy
Edit
./gradlew assembleDebug